In [2]:
import pandas as pd
import pickle

data = pd.read_pickle('dtm_stop.pkl')


In [5]:
from gensim import matutils, models
import scipy.sparse
tdm = data.transpose()

,ali,anthony,bill,bo,dave,hasan,jim,joe,john,louis,mike,ricky
aaaaah,0,0,1,0,0,0,0,0,0,0,0,0


In [9]:
spr_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(spr_counts)

cv = pickle.load(open('cv_stop.pkl', 'rb'))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())
lda = models.LdaModel(corpus = corpus, id2word = id2word, num_topics = 2, passes = 10)


[(0,
  '0.006*"fucking" + 0.006*"say" + 0.005*"didnt" + 0.005*"theyre" + 0.005*"fuck" + 0.005*"cause" + 0.005*"shit" + 0.005*"really" + 0.004*"thing" + 0.004*"good"'),
 (1,
  '0.009*"fucking" + 0.006*"shit" + 0.006*"want" + 0.006*"going" + 0.005*"fuck" + 0.005*"love" + 0.005*"hes" + 0.004*"day" + 0.004*"say" + 0.004*"theyre"')]

In [44]:
from nltk import word_tokenize, pos_tag

def nouns(text):
    noun = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if noun(pos)]
    return ' '.join(all_nouns)

In [45]:
data_clean = pd.read_pickle('data_clean.pkl')
data_nouns = pd.DataFrame(data_clean.transcript.apply(nouns))
data_nouns

,transcript
ali,ladies gentlemen welcome stage ali wong hi wel...
anthony,thank san francisco thank good people surprise...
bill,right thank thank pleasure greater atlanta geo...
bo,old macdonald farm e i i o farm pig e i i snor...
dave,dirty jokes living stare most hard work profou...
hasan,whats davis whats im home i netflix special la...
jim,ladies gentlemen welcome stage mr jim jefferie...
joe,ladies gentlemen joe fuck san francisco thanks...
john,right petunia august thats good right hello he...
louis,music lets lights lights thank much i i i nice...


In [51]:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer
add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)
cv = CountVectorizer(stop_words = stop_words, max_df = .8)
data_cvn = cv.fit_transform(data_nouns.transcript)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns = cv.get_feature_names_out())
data_dtmn.index = data_nouns.index
data_dtmn

,aaaaah,aaaaahhhhhhh,aaaaauuugghhhhhh,aaaahhhhh,aah,abc,abcs,ability,abject,able,...,ze,zealand,zee,zeppelin,zero,zillion,zombie,zombies,zoo,éclair
ali,0,0,0,0,0,1,0,0,0,2,...,0,0,0,0,0,0,1,0,0,0
anthony,0,0,0,0,0,0,0,0,0,0,...,0,10,0,0,0,0,0,0,0,0
bill,1,0,0,0,0,0,1,0,0,1,...,1,0,0,0,0,1,1,1,0,0
bo,0,1,1,1,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
dave,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
hasan,0,0,0,0,0,0,0,0,0,1,...,0,0,2,0,0,0,0,0,0,0
jim,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
joe,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
john,0,0,0,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,1
louis,0,0,0,0,3,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [52]:
corpush = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))
id2words = dict((v, k) for k, v in cv.vocabulary_.items())
ldan = models.LdaModel(corpus=corpush, num_topics=4, id2word=id2words, passes=80)
ldan.print_topics()

[(0,
  '0.010*"joke" + 0.009*"bo" + 0.007*"repeat" + 0.007*"anthony" + 0.006*"eye" + 0.006*"jokes" + 0.006*"contact" + 0.005*"comedy" + 0.005*"cos" + 0.005*"grandma"'),
 (1,
  '0.005*"joke" + 0.005*"ahah" + 0.004*"husband" + 0.003*"gay" + 0.003*"nigga" + 0.003*"nuts" + 0.003*"jenner" + 0.003*"ok" + 0.003*"rape" + 0.003*"asian"'),
 (2,
  '0.004*"gun" + 0.004*"class" + 0.004*"parents" + 0.004*"hasan" + 0.004*"jenny" + 0.003*"guns" + 0.003*"son" + 0.003*"door" + 0.003*"ass" + 0.003*"mom"'),
 (3,
  '0.005*"mom" + 0.005*"clinton" + 0.004*"president" + 0.004*"dick" + 0.004*"parents" + 0.004*"tit" + 0.004*"cow" + 0.003*"wife" + 0.003*"dog" + 0.003*"ha"')]

In [53]:
corpus_new = ldan[corpush]
list(zip([a for[(a, b)] in corpus_new], data_dtmn.index))

[(1, 'ali'),
 (0, 'anthony'),
 (2, 'bill'),
 (0, 'bo'),
 (1, 'dave'),
 (2, 'hasan'),
 (2, 'jim'),
 (3, 'joe'),
 (3, 'john'),
 (3, 'louis'),
 (2, 'mike'),
 (1, 'ricky')]